In [1]:
import numpy as np
import matplotlib.pyplot as plt
import numexpr as ne
import scipy.signal as signal
import time
from numba import jit
%matplotlib qt5

In [2]:
from tools import sb1, wa, pulse, pulse2

# Backprojection

## Load the analytic signal

In [4]:
data_dir = "/home/pleroy/DATA/SIMU"
analyticSignal = np.load( data_dir + "/analyticSignal_0_19.npy" )
print( "analyticSignal.shape = {}".format(analyticSignal.shape) )

analyticSignal.shape = (30000, 1500)


## Range focalization

In [5]:
srf = np.fft.ifft( analyticSignal, axis=1 )
print( "srf.shape = {}".format(srf.shape) )

srf.shape = (30000, 1500)


# Image geometry

In [10]:
x_min = 200
x_max = 500
r_min = 80
r_max = 120

#extent=[horizontal_min,horizontal_max,vertical_min,vertical_max]
extent=[r_min,r_max,x_max,x_min]

d_x = 1
d_r = 1

x = np.arange( x_min, x_max, d_x )
x = x.reshape( x.size, 1 )
r = np.arange( r_min, r_max, d_r )
r = r.reshape( 1, r.size )

## Azimuth focalization

In [16]:
B = 150e6
c = 3e8
nbFiles = 20
fc = 5.8e9
kc = 4 * np.pi * fc / c
# vehicule is plane
T = 600e-6
rampsPerFile = 1500
V = 40

In [17]:
overSamplingRatio = 10
Nf = srf.shape[1]
Nover = overSamplingRatio * Nf
rangeResolution = c / (2 * B)
r_base = np.arange( Nf ) * rangeResolution
r_over = np.arange( Nover ) * rangeResolution / overSamplingRatio
print( "range from {:.2f}m to {:.2f}m, resolution = {}m, oversampled = {}m, ".format(
    r_over[0], r_over[-1], rangeResolution, rangeResolution / overSamplingRatio ) )

range from 0.00m to 1499.90m, resolution = 1.0m, oversampled = 0.1m, 


In [18]:
Naz = nbFiles * rampsPerFile
xa_vec = np.arange( Naz ) * T * V

print( "x.shape = {}, r.shape = {}".format( x.shape, r.shape ) )
print( "x from {} to {}, r from {} to {}".format(x[0,0], x[-1,0], r[0,0], r[0,-1]) )

phi = 6 * np.pi / 180

nbLoops = xa_vec.shape[0]
progress = int( nbLoops / 10 )

x.shape = (300, 1), r.shape = (1, 40)
x from 200 to 499, r from 80 to 119


In [20]:
t = time.time()

img  = np.zeros( (x.size, r.size), dtype=complex )
print( "img.shape = {}".format( img.shape ) )

progress = int( xa_vec.size / 10 )

loop = 0
endK = xa_vec.size
for k in range( endK ):
    xa =xa_vec[k]
    loop = k
    if xa in xa_vec[::progress]:
        print( "{:.2f} / {:.2f}".format(xa, xa_vec[-1] ) )

    if k == endK-1:
        d = (r[0,20]**2 + (xa-x[160,0])**2 )**0.5
        aux1 = np.exp( 1j * kc * d )
        y = signal.resample( srf[loop,:], Nover  )
        aux2 = np.interp( d, r_over, signal.resample( srf[loop,:], Nover  ) )
        aux3 = pulse2( (xa-x[160,0]) / (r[0,20]*np.tan(phi)) )
        aux4 = aux1 * aux2 * aux3
        print("k = {}   d = {}   aux1 = {}   aux2 = {}\n   aux3 = {}   aux4 = {}".format( k, d, aux1, aux2, aux3, aux4) )
        print("srf = {:.10f}   srf = {:.10f}\ny = {:.10f}   y = {:.10f}\n".format(srf[k, 0], srf[k, 1], y[1000], y[1001]))

    img += np.exp( 1j * kc * (r**2 + (xa-x)**2 )**0.5 ) \
    * np.interp( (r**2 + (xa-x)**2 )**0.5, r_over, signal.resample( srf[loop,:], Nover  ) ) \
    * pulse( (xa-x) / (r*np.tan(phi)) )
    
    loop += 1
# np.exp( -1j * kc * r)

elapsed = time.time() - t
print("execution time = " + str(elapsed))

img.shape = (300, 40)
0.00 / 719.98
72.00 / 719.98
144.00 / 719.98
216.00 / 719.98
288.00 / 719.98
360.00 / 719.98
432.00 / 719.98
504.00 / 719.98
576.00 / 719.98
648.00 / 719.98
k = 29999   d = 373.60770947077623   aux1 = (0.5103052855040465+0.8599933229895647j)   aux2 = (-0.011301688611412562-0.006427530147536355j)
   aux3 = 0   aux4 = -0j
srf = 0.0000090742+0.0000337036j   srf = 0.0000097313+0.0000336945j
y = 0.0000209429+0.0000402568j   y = 0.0000278229+0.0000534817j

execution time = 87.38529777526855


In [21]:
img[0,0]

(2.2391811659142823e-05+1.1432034209496042e-05j)

In [ ]:
plt.figure()
plt.imshow( 20 * np.log10( np.abs( img ) ), cmap='jet', extent=extent )
plt.grid()
plt.colorbar(orientation="horizontal")

# fft2d

In [ ]:
ifft_adc = np.fft.ifft( adc[::2,:], axis=1 )

In [ ]:
plt.figure()
plt.imshow(20*np.log10(np.abs(ifft_adc[::10, :])))
plt.colorbar(orientation='horizontal')

In [ ]:
fft2d_adc = np.fft.fftshift( np.fft.fft( ifft_adc, axis=0 ), axes = 0 )

In [ ]:
plt.figure()
plt.imshow(20*np.log10(np.abs(fft2d_adc[::10, :])))
plt.colorbar(orientation='horizontal')

In [ ]:
plt.figure()
plt.plot( 20 * np.log10(np.abs( fft2d_adc[:,50] )) )
plt.plot( 20 * np.log10(np.abs( fft2d_adc[:,59] )) )

### Resample

In [3]:
plt.imshow( 20 * np.log10( np.abs( srf[::10, :] ) ) )

NameError: name 'srf' is not defined

In [4]:
nbPointsResampled = 4 * srf.shape[1]
srf_resampled = signal.resample( srf[0,:], nbPointsResampled  )
print( "srf.shape = {}, srf_resamples.shape = {}".format( srf.shape, srf_resampled.shape ) )

NameError: name 'srf' is not defined

In [5]:
up_vec  = np.zeros( nbPointsResampled, dtype = complex )
vec_RD  = np.zeros( nbPointsResampled, dtype = complex  )

Nf = srf.shape[1]

vec_ind = int( np.ceil( ( Nf + 1 ) / 2 ) )

pos = 0
#upsample raw data
vec_RD[ 0 : vec_ind ] = adc[ pos, 0 : vec_ind ]
vec_RD[ - ( Nf - vec_ind ) : ] = adc[ pos, vec_ind : ]
# vec_RD contains RD(pos,:) and zeros: up-sampling

up_vec = np.fft.ifft( vec_RD ) # Transformation to time domain

print( "adc.shape = {}, vec_ind = {}, vec_RD.shape = {}".format( adc.shape, vec_ind, vec_RD.shape ) )

NameError: name 'nbPointsResampled' is not defined

In [ ]:
plt.figure()
plt.subplot(211)
plt.plot( np.arange( srf.shape[1] ), np.real( srf[0,:] ), 'go-' )
plt.plot( np.arange( nbPointsResampled ) / 4, np.real( srf_resampled ), 'b.-' )
plt.plot( np.arange( nbPointsResampled ) / 4, np.real( up_vec ) * 4, '.-', color="pink" )
ax = plt.gca()
plt.subplot(212, sharex=ax)
plt.plot( np.arange( srf.shape[1] ), np.imag( srf[0,:] ), 'go-' )
plt.plot( np.arange( nbPointsResampled ) / 4, np.imag( srf_resampled ), 'b.-' )
plt.plot( np.arange( nbPointsResampled ) / 4, np.imag( up_vec ) * 4, '.-', color="pink" )